In [1]:
## Apriori algorithm
import pandas as pd
from pandas import DataFrame
import itertools
import numpy as np

In [2]:
def conv(val):
        if val == 't':
            return 1
        else:
            return 0
        
class CSVLoader():
    def __init__(self) -> None:
        pass
    # load csv

    def load_csv(self, csv_name: str) -> DataFrame:
        self.column_list = pd.read_csv(csv_name, index_col=0, nrows=0).columns.tolist()
        csv_header_len = len(self.column_list) + 1
        csv_file = pd.read_csv(csv_name, engine="python", converters={k: conv for k in range(csv_header_len)})
        return csv_file

In [ ]:
class Apriori():

    def __init__(self, support: float, confidence: float, csv_name: str) -> None:
        csv_loader = CSVLoader()
        self.data = csv_loader.load_csv(csv_name=csv_name)
        self.min_support = support
        self.confidence = confidence
        self.row_count = len(self.data.index)

    def calculate_support(self, data: DataFrame) -> dict:
        support_count = {item:0 for item in data.columns}
        for column in data.columns:
            support_count[column] = data[column].value_counts()[1]/self.row_count
        return support_count

    def prune(self, current_combinations: list, previous_frequent_combinations: list, previous_length: int) -> list:
        # remove all items from set that are not part of previous frequent list
        # combinations = [(1,2,3), ... , (1,2,3)]
        # if previous_length == 1:
        #     print("simple 2 element combinations checked")
        #     return current_combinations
        # else:
        new_combinations_list = current_combinations.copy()
        previous_frequent_combinations = [set(item) for item in previous_frequent_combinations]
        for index, item in enumerate(current_combinations):
            temp_combinations = list(itertools.combinations(item, previous_length))
            for prev_item in temp_combinations:
                temp_set = set(prev_item)
                print(not temp_set in previous_frequent_combinations)
                if not temp_set in previous_frequent_combinations:
                    new_combinations_list.remove(item)
                    break        
            return new_combinations_list

    def above_min_support(self, frequency_dict: dict) -> list:
        #above_min_support = []
        #for key in frequency_dict.keys():
        #  if frequency_dict[key] > self.min_support:
        #     above_min_support.append(key)
        above_min_support = [{key: support} for key, support in frequency_dict.items() if support > self.min_support]
        return above_min_support

    def apriori(self) -> list:
        #Run apriori
        #read file and format
        c1 = self.calculate_support(self.data)
        l1 = self.above_min_support(c1)
        print(l1)

        accumulated_frequent_dataset = {}

        current_lset = l1
        current_iteration = 2
        test = True
        while current_lset and test:
            #store frequent item set
            accumulated_frequent_dataset[current_iteration - 1] = current_lset
            print(len(current_lset))
            # create combinations
            current_lset = list(np.array(item.keys() for item in current_lset).ravel())
            combinations = list(itertools.combinations(current_lset, current_iteration))
            # print(len(combinations))
            # prune
            combinations = self.prune(current_combinations=combinations, previous_frequent_combinations=current_lset, previous_length=current_iteration-1)
            # calculate above min support
            current_c = {}
            count = 0
            print(combinations)
            for item in combinations:
                condition = np.all(self.data[list(item)], axis=1)
                count = np.sum(condition)
                current_c[item] = count/self.row_count
                # print(current_c)
                count = 0
            current_lset = self.above_min_support(current_c)
            print(len(current_lset))
            # current lset from here
            current_iteration += 1
            if current_iteration > 5:
                test = False
            test = False
        return []# list of association rules

def main():
    support = 0.005
    confidence = 0.6
    csv_name = "./myDataFile.csv"
    apriori = Apriori(support=support, confidence=confidence, csv_name=csv_name)
    association_rules = apriori.apriori()
    # print(association_rules)

if __name__=="__main__":
    main()

120
True
[('Instant_food_products', 'baking_powder'), ('Instant_food_products', 'beef'), ('Instant_food_products', 'berries'), ('Instant_food_products', 'beverages'), ('Instant_food_products', 'bottled_beer'), ('Instant_food_products', 'bottled_water'), ('Instant_food_products', 'brown_bread'), ('Instant_food_products', 'butter'), ('Instant_food_products', 'butter_milk'), ('Instant_food_products', 'cake_bar'), ('Instant_food_products', 'candles'), ('Instant_food_products', 'candy'), ('Instant_food_products', 'canned_beer'), ('Instant_food_products', 'canned_fish'), ('Instant_food_products', 'canned_vegetables'), ('Instant_food_products', 'cat_food'), ('Instant_food_products', 'cereals'), ('Instant_food_products', 'chewing_gum'), ('Instant_food_products', 'chicken'), ('Instant_food_products', 'chocolate'), ('Instant_food_products', 'chocolate_marshmallow'), ('Instant_food_products', 'citrus_fruit'), ('Instant_food_products', 'cleaner'), ('Instant_food_products', 'cling_film_bags'), ('In

: 